In [16]:
import json

In [2]:
import pandas as pd

In [3]:
filter_index = ['京-聲韻', '京-聲韻調', '粵-聲', '粵-韻', '粵-調', '粵-聲韻調', '字頭', '呼', '等', '韻部', '調', '聲類', '攝', '中古拼音(polyhedron 版)','轉簡']

In [4]:
df = pd.read_excel('./粵音小鏡(20160723).xls', sheet_name= 3,header=0, usecols=filter_index)

In [5]:
with open('../总成语表.json', 'r', encoding='utf-8') as file:
    dict_idiom = json.load(file)

In [1]:
import opencc

In [2]:
converter = opencc.OpenCC('s2t.json')

In [6]:
converter.convert('群')

'羣'

In [9]:
from typing import Tuple

def translate(pair: Tuple[str, str]):
    char, pinyin = pair
    item = df.query("`字頭` == '{}'".format(char))
    if (item.shape[0] == 0):
        # print('Not found '+char)
        return item
    elif (item.shape[0] == 1):
        return item
    else:
        item = item.query("`京-聲韻調`=='{}'".format(pinyin))
        return item

In [20]:
from tqdm import tqdm

In [8]:
import re

In [12]:
def style_transfer(pinyin: str):

    pinyin = re.sub(re.compile(r'^(y|j|q|x)u([a-z]*[0-9]?)$'), r'\1v\2',pinyin)
    pinyin =  re.sub(re.compile(r'y(i?)([^v])'),r'yi\2',pinyin)
    pinyin =  re.sub(re.compile(r'wu?'),'wu',pinyin)
    pinyin = pinyin.replace('v', 'ü')
    pinyin =  re.sub(re.compile(r'^(b|p|m|f)o([0-9]?)$'), r'\1uo\2',pinyin)
    pinyin = pinyin.replace(r'iu', 'iou').replace(r'ui', 'uei').replace(r'un', 'uen')
    
    return pinyin

In [13]:
print(style_transfer('ju'))
print(style_transfer('lu'))
print(style_transfer('yu'))
print(style_transfer('yao'))
print(style_transfer('liu'))
print(style_transfer('yun'))
print(style_transfer('lun'))
print(style_transfer('kui'))
print(style_transfer('wei'))
print(style_transfer('weng'))
print(style_transfer('yong'))
print(style_transfer('xiu'))

jü
lu
yü
yiao
liou
yün
luen
kuei
wuei
wueng
yiong
xiou


In [14]:
import sys

In [60]:
import pingyam_converter as py_converter

In [63]:
dict_idiom_yueping = {}
heteronym = {}
no_yueping = {}
for (idiom, phone) in tqdm(dict_idiom.items()):
    idiom_trad = converter.convert(idiom)
    pinyins = phone.split(' ')
    pingyams = []
    
    for index, char in enumerate(idiom_trad):
        pinyin = pinyins[index]
        transfered_pinyin = style_transfer(pinyin)
        item = translate((char,transfered_pinyin))["粵-聲韻調"]
        # try:
        #     pingyam = py_converter.converter(pingyam, from_style='广州拼音', to_style='粤拼')
        # except:
        #     print(pingyam)
        if (item.shape[0] != 0):    
            pingyam = item.iloc[0]
            pingyams.append(pingyam)
            if(item.shape[0] > 1):
                heteronym.update({char: pingyam})
        else:
            pingyams.append('?')
            no_yueping.update({char: pingyam})
            # sys.stdout.write("%s \t %s" % (len(no_yueping), {char: (pinyin, transfered_pinyin) }))
            # sys.stdout.flush()
        
    dict_idiom_yueping[idiom] = ' '.join(pingyams)


100%|██████████| 29738/29738 [11:22<00:00, 43.58it/s]


In [64]:
#文白异读
len(heteronym)

217

In [65]:
with open('./粤语成语表（广州拼音）.json', 'w', encoding='utf-8') as outfile:
    json.dump(dict_idiom_yueping,outfile,  ensure_ascii=False, indent=0)

In [13]:
data = pd.read_csv('./pingyambiu.txt', sep='\t', header=None, usecols=[6, 7])
data.columns = ['粤拼', '广州拼音']

In [42]:
import re

In [84]:
with open('./粤语成语表（广州拼音）.json', 'r', encoding='utf-8') as file:
    txt = file.read()

In [89]:
#使用香港学会标准
for ind in tqdm(data.index):
    old = re.compile('(?<=[^a-z]){}(?=[^a-z])'.format(data.iloc[ind]['广州拼音']))
    new = data.iloc[ind]['粤拼']
    txt = re.sub(old, new, txt)
    # break

100%|██████████| 3946/3946 [02:14<00:00, 29.40it/s]


In [90]:
with open('./粤语成语表（香港拼音）.json', 'w', encoding='utf-8') as outfile:
    outfile.write(txt)

In [83]:
old1 = re.compile(r'([" ]){}([ "])'.format('jing1'), flags=re.M)
print(re.sub(old1, r"\1{}\2".format('ging1'), 'jing1 jing1 yig6 yig6'))

old2 = re.compile(r'([" ]){}([ "])'.format('yig6'))
print(re.sub(old2, r"\1{}\2".format('jik6'), 'ging1 ging1 yig6 yig6'))

jing1 ging1 yig6 yig6
ging1 ging1 jik6 yig6


### fix customized pingyam style in `粤音小镜.json`

In [9]:
idioms = {}
phones = set()
with open('./粤语成语表（香港拼音）.json', 'r', encoding='utf-8') as file:
    for line in file:
        if '?' in line:
            idiom, phone = line.strip('\n').replace('\"','').replace(',','').split(': ')
            phone_list = phone.split(' ')
            phones = phones.union(set(phone_list))
            idioms.update({idiom: phone_list})


with open('./粤语成语表（香港拼音）.json', 'r', encoding='utf-8') as file:
    txt = file.read()


In [22]:
txt[:100]

'{\n    "一世之雄": "jat1 sai3 zi1 hung4",\n    "一世英名": "jat1 sai3 jing1 ming4",\n    "一世龙门": "jat1 sai3 lun'

In [23]:
phones_to_be_normalized = []
for phone in phones:
    # phone = phone[:-1]
    if not phone in data['粤拼'].unique().tolist():
        phones_to_be_normalized.append(phone)

In [24]:
phones_to_be_normalized

['xun5',
 'ju2',
 'yêu5',
 'ju6',
 'guad3',
 'sêu3',
 'jun1',
 'gêu3',
 'qun3',
 'yu4',
 'guo2',
 'hêu1',
 'yun3',
 'lêu5',
 'guai1',
 'xud3',
 'kuang1',
 'xun4',
 'xu4',
 'qu2',
 'guei6',
 'sêu5',
 'guong1',
 'xu2',
 'gua3',
 'cêu4',
 'yêu6',
 'hêu3',
 'guan1',
 'guei1',
 'gued1',
 'cêu1',
 'lêu6',
 'yun4',
 'kêu1',
 'yun1',
 'têu1',
 'zêu2',
 'yun5',
 'dêu3',
 'xun2',
 'jun2',
 'kua1',
 'gua2',
 'yud6',
 'guei2',
 'zêu6',
 'gueng1',
 'lêu3',
 'kuen4',
 'jud3',
 'sêu2',
 'gêu2',
 'yu6',
 'xun1',
 'cêu2',
 'jud6',
 'kuen3',
 'ju3',
 'jun3',
 'zêu1',
 'qud3',
 'qu5',
 'yu1',
 'guo3',
 'xu6',
 'guo1',
 'lêu4',
 'kuei1',
 'gêu1',
 'gua1',
 'guig1',
 'gued6',
 'guog3',
 'qun4',
 'qu4',
 'qun1',
 'guei3',
 'guai2',
 'têu3',
 'guai3',
 'sêu4',
 'nêu5',
 'yu5',
 'kuong3',
 'yun2',
 'xu1',
 'cêu3',
 'ju1',
 'guen1',
 'gêu6',
 'sêu6',
 '?']

In [25]:
#使用香港学会标准
phones_to_be_normalized_remained = []
for phone in tqdm(phones_to_be_normalized):
    old = re.compile('(?<=[^a-z]){}(?=[^a-z])'.format(phone))
    try:
        new = data[data['广州拼音'] == phone]['粤拼'].iloc[0]
    except:
        tmpl = re.compile(r'(k|g)u')
        phone_u_to_w = re.sub(tmpl, r'\1w', phone)
        try:
            new = data[data['广州拼音'] == phone_u_to_w]['粤拼'].iloc[0]
        except:
            phones_to_be_normalized_remained.append(phone)
            break
    txt = re.sub(old, new, txt)
    # break

 99%|█████████▉| 92/93 [00:04<00:00, 18.95it/s]


In [27]:
txt[:110]

'{\n    "一世之雄": "jat1 sai3 zi1 hung4",\n    "一世英名": "jat1 sai3 jing1 ming4",\n    "一世龙门": "jat1 sai3 lung4 mun4",\n'

In [28]:
phones_to_be_normalized_remained

['?']

In [29]:
with open('./粤语成语表（香港拼音）.json', 'w', encoding='utf-8') as outfile:
    outfile.write(txt)

In [30]:
len(idioms)

2539

In [3]:
import sqlite3

In [4]:
path = './mcpdict.db'
conn = sqlite3.connect(path)

In [6]:
cursor = conn.cursor()

In [ ]:
cursor.execute()

In [11]:
def lookup_db(cursor, unicode):
    cursor.execute('''SELECT ct FROM mcpdict WHERE unicode=?''', (unicode,))
    row = cursor.fetchone()
    return row

In [26]:
with open('./粤语成语表（香港拼音）.json', 'r', encoding='utf-8') as file:
    idioms = json.load(file)

In [27]:
for (idiom, phones) in tqdm(idioms.items()):
    phone_list = phones.split(' ')
    if '?' in phone_list:
        indices = [i for i,k in enumerate(phone_list) if k == '?']
        for ind in indices:
            char = converter.convert(idiom[ind])
            unicode_ord = str(hex(ord(char)))[2:].upper()
            phone = lookup_db(cursor, unicode_ord)
            if phone[0] == None:
                phone_list[ind] = '??'
            else:
                phone_list[ind] = phone[0]
        idioms[idiom] = ' '.join(phone_list)


100%|██████████| 29738/29738 [00:33<00:00, 877.11it/s] 


In [29]:
with open('./粤语成语表（香港拼音）.json', 'w', encoding='utf-8') as outfile:
    json.dump(idioms, outfile, ensure_ascii=False, indent=0)

In [30]:
conn.close()

In [23]:
phone_list

['haa6', None, 'zau2', 'jyun4']

In [24]:
phone

(None,)

In [25]:
char

'坂'

In [18]:
type(phone)

tuple

In [40]:
unicode_raw.s

b'\\u9a6a'


In [37]:
idiom

'    一串骊珠'

In [33]:
"jat1 ? jat1 ?".split(' ').index('?')

1

### fix no pronounciation characters by mcpdict database